# Raster Match

By Cascade Tuholske, June 2020 <br>

Notebook reprojects popgrid rasters into the same CRS, size and projection so they stack.

Updated CPT Sep 2020 to set all meta data to GPWv4 <br>
Replaces Raster Match NB Dec 2020<br>
Updated Feb 2023 by CPT

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio 
import rioxarray as rio
from glob import glob
from rasterio.enums import Resampling
import os

In [3]:
# file paths 
data_in = os.path.join('../data/raw/')
data_out = os.path.join('../data/processed/')

## Functions

In [4]:
def resample(fn_in, fn_out, scale_factor, method):
    
    """ Resamples a raster and save it out
    Args:
        fn_in = file path and name of tif input as str
        fn_out = file path and name of tif output as str 
        scale_factor = factor to up or down scale a pixel as float
        method = method to resample (rasterio object), see rasterio documentation
    """
    
    with rasterio.open(fn_in) as dataset:

        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=method
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
    
    # meta data to write out
    out_meta = dataset.meta

    # Update meta data
    out_meta.update({"driver": "GTiff",
             "height": data.shape[1],
             "width": data.shape[2],
             "transform": transform})

    # write image 
    with rasterio.open(fn_out, "w", **out_meta) as dest:
        dest.write(data)

## Open files and check meta data

In [6]:
# input file paths
wp_path = os.path.join(data_in, 'Worldpop-unconstrained/ppp_2020_1km_Aggregated.tif')
ls_path = os.path.join(data_in, 'landscan-global-2020-assets/landscan-global-2020.tif')
ghs_path = os.path.join(data_in, 'GHS_POP_E2020_GLOBE_R2022A_54009_1000_V1_0/GHS_POP_E2020_GLOBE_R2022A_54009_1000_V1_0.tif')
gpw_path = os.path.join(data_in, 'gpw-v4-population-count-rev11_2020_30_sec_tif/gpw_v4_population_count_rev11_2020_30_sec.tif')

In [7]:
# GPW v4 : width = 43200 height = 21600 epsg = 4326
gpw = rio.open_rasterio(gpw_path)
gpw

<xarray.DataArray (band: 1, y: 21600, x: 43200)>
[933120000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    DataType:            Generic
    RepresentationType:  ATHEMATIC
    _FillValue:          -3.402823e+38
    scale_factor:        1.0
    add_offset:          0.0

In [8]:
# World Pop 2016 (?_): width = 43200 height = 18720 epsg = 4326
wp = rio.open_rasterio(wp_path)
wp

<xarray.DataArray (band: 1, y: 18720, x: 43200)>
[808704000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 84.0 83.99 83.98 83.97 ... -71.98 -71.99 -72.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -3.4028235e+38
    scale_factor:   1.0
    add_offset:     0.0

In [10]:
# LS 2015 : width = 43200 height = 21600 epsg = 4326
ls = rio.open_rasterio(ls_path)
ls

<xarray.DataArray (band: 1, y: 21600, x: 43200)>
[933120000 values with dtype=int32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -2147483647
    scale_factor:   1.0
    add_offset:     0.0

In [11]:
# GHS 2015 : width = 43200 height = 21600 epsg = 4326
ghs = rio.open_rasterio(ghs_path)
ghs

<xarray.DataArray (band: 1, y: 18000, x: 36082)>
[649476000 values with dtype=float64]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -1.804e+07 -1.804e+07 ... 1.804e+07 1.804e+07
  * y            (y) float64 9e+06 8.998e+06 8.998e+06 ... -8.998e+06 -9e+06
    spatial_ref  int64 0
Attributes: (12/13)
    AREA_OR_POINT:             Area
    STATISTICS_COVARIANCES:    330985.3282731281
    STATISTICS_MAXIMUM:        615756.8270821
    STATISTICS_MEAN:           56.450798856297
    STATISTICS_MEDIAN:         0.0
    STATISTICS_MINIMUM:        0
    ...                        ...
    STATISTICS_SKIPFACTORY:    1
    STATISTICS_STDDEV:         575.31324361006
    STATISTICS_VALID_PERCENT:  21.26
    _FillValue:                -200.0
    scale_factor:              1.0
    add_offset:                0.0

# Reset zero values

In [13]:
print('GPW na value is', gpw.data[0][0][0])
print('World Pop na value is', wp.data[0][0][0])
print('LandScan na value is', ls.data[0][0][0])
print('GHS na value is', ghs.data[0][0][0])

GPW na value is -3.402823e+38
World Pop na value is -3.4028235e+38
LandScan na value is -2147483647
GHS na value is -200.0


In [15]:
# Set all NA values to -9999
gpw.data = np.where(gpw.data < 0, -9999, gpw.data)
wp.data = np.where(wp.data < 0,-9999, wp.data)
ls.data = np.where(ls.data < 0,-9999, ls.data)
ghs.data = np.where(ghs.data < 0,-9999, ghs.data)


In [16]:
print('GPW na value is', gpw.data[0][0][0])
print('World Pop na value is', wp.data[0][0][0])
print('LandScan na value is', ls.data[0][0][0])
print('GHS na value is', ghs.data[0][0][0])

GPW na value is -9999.0
World Pop na value is -9999.0
LandScan na value is -9999
GHS na value is -9999.0


## Matching and Reprojecting

trying: https://corteva.github.io/rioxarray/html/examples/reproject_match.html

In [17]:
# Check NA values
print(gpw.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ghs.attrs['_FillValue'])

-3.402823e+38
-2147483647
-3.4028235e+38
-200.0


In [19]:
# Change fill value to - 9999 
na_val = -9999
gpw.attrs['_FillValue'] = na_val
ls.attrs['_FillValue'] = na_val
wp.attrs['_FillValue'] = na_val
ghs.attrs['_FillValue'] = na_val

In [20]:
# Check NA values 
print(gpw.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ghs.attrs['_FillValue'])

-9999
-9999
-9999
-9999


In [22]:
# Reproject LS, WP, and GHS datasets to GPWv4
ls_match = ls.rio.reproject_match(gpw)
wp_match = wp.rio.reproject_match(gpw)
ghs_match = ghs.rio.reproject_match(gpw)

In [23]:
# Check NA values
print(gpw.attrs['_FillValue'])
print(ls_match.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ghs_match.attrs['_FillValue'])

-9999
-9999
-9999
-9999.0


## Check Final

In [24]:
gpw

<xarray.DataArray (band: 1, y: 21600, x: 43200)>
array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],
      dtype=float32)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    DataType:            Generic
    RepresentationType:  ATHEMATIC
    _FillValue:          -9999
    scale_factor:        1.0
    add_offset:          0.0

In [25]:
ls_match

<xarray.DataArray (band: 1, y: 21600, x: 43200)>
array([[[-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0]]], dtype=int32)
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     -9999

In [26]:
wp_match

<xarray.DataArray (band: 1, y: 21600, x: 43200)>
array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],
      dtype=float32)
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     -9999.0

In [27]:
ghs_match

<xarray.DataArray (band: 1, y: 21600, x: 43200)>
array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]])
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes: (12/13)
    AREA_OR_POINT:             Area
    STATISTICS_COVARIANCES:    330985.3282731281
    STATISTICS_MAXIMUM:        615756.8270821
    STATISTICS_MEAN:           56.450798856297
    STATISTICS_MEDIAN:         0.0
    STATISTICS_MINIMUM:        0
    ...                        ...
    STATISTICS_SKIPFACTORY:    1
    STATISTICS_STDDEV:         575.31324361006
    STATISTICS_VALID_PERCENT:  21.26
    scale_factor:              1.0
    add_offset:                0.0
    _FillValue:                -9999.0

In [28]:
# Try writing them and then checking them
gpw.rio.to_raster(data_out+'GPWv4-2020-matched.tif')
ls_match.rio.to_raster(data_out+'LS-2020-matched.tif')
wp_match.rio.to_raster(data_out+'WP-2020-matched.tif')
ghs_match.rio.to_raster(data_out+'GHS-2020-matched.tif')